+ id：标识号
+ imdb_id：IMDB 标识号
+ popularity：在 Movie Database 上的相对页面查看次数
+ budget：预算（美元）
+ revenue：收入（美元）
+ original_title：电影名称
+ cast：演员列表，按 | 分隔，最多 5 名演员
+ homepage：电影首页的 URL
+ director：导演列表，按 | 分隔，最多 5 名导演
+ tagline：电影的标语
+ keywords：与电影相关的关键字，按 | 分隔，最多 5 个关键字
+ overview：剧情摘要
+ runtime：电影时长
+ genres：风格列表，按 | 分隔，最多 5 种风格
+ production_companies：制作公司列表，按 | 分隔，最多 5 家公司
+ release_date：首次上映日期
+ vote_count：评分次数
+ vote_average：平均评分
+ release_year：发行年份
+ budget_adj：根据通货膨胀调整的预算（2010 年，美元）
+ revenue_adj：根据通货膨胀调整的收入（2010 年，美元）

问题1：电影类型是如何随着时代变化而变化的?
问题2：环球影业和派拉蒙影业的电影之间的数据指标有什么区别?
问题3：和非小说改编的电影相比，基于小说改编的电影表现得怎么样?

In [27]:
import numpy as np
import pandas as pd
filename = './movies.csv'
movies_df = pd.read_csv(filename, encoding='utf-8')
movies_df.describe()


,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,10866.000000,10866.000000,1.086600e+04,1.086600e+04,10866.000000,10866.000000,10866.000000,10866.000000,1.086600e+04,1.086600e+04
mean,66064.177434,0.646441,1.462570e+07,3.982332e+07,102.070863,217.389748,5.974922,2001.322658,1.755104e+07,5.136436e+07
std,92130.136561,1.000185,3.091321e+07,1.170035e+08,31.381405,575.619058,0.935142,12.812941,3.430616e+07,1.446325e+08
min,5.000000,0.000065,0.000000e+00,0.000000e+00,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00
25%,10596.250000,0.207583,0.000000e+00,0.000000e+00,90.000000,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00
50%,20669.000000,0.383856,0.000000e+00,0.000000e+00,99.000000,38.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00
75%,75610.000000,0.713817,1.500000e+07,2.400000e+07,111.000000,145.750000,6.600000,2011.000000,2.085325e+07,3.369710e+07
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,900.000000,9767.000000,9.200000,2015.000000,4.250000e+08,2.827124e+09


In [28]:
# 由于一个电影由多个电影公司所制作，所以对电影公司这一列进行拆分。

def split_date(data, column) :
    split_genres = data[column].str.split('|', expand=True)

    split_genres['id'] = data['id']
    merged_back = data.merge(split_genres)
    merged_back.drop_duplicates(subset=None, keep='first', inplace=True)
    
    #得到原数据集所有columns
    columns = data.columns.values.tolist()
    #columns去除要分割的列名
    columns.remove(column)
    
    melted = pd.melt(
        merged_back, id_vars=columns, 
        value_vars=[0, 1, 2, 3, 4], value_name=column+'_only'
    ).drop('variable', axis = 1
    ).dropna()
    
    return melted

data_split_genres = split_date(movies_df, 'genres')
test_movies = split_date(data_split_genres, 'production_companies')
test_movies.to_excel('test_movies.xlsx', encoding='utf-8', index=False)
# data_split_genres.to_excel('data_split_genres.xlsx', encoding='utf-8', index=False)# 由于一个电影由多个电影公司所制作，所以对电影公司这一列进行拆分。


In [29]:
# 将电影公司进行拆分，仅留下环球影业和派拉蒙影业
movies = pd.read_csv('movies.csv', encoding='utf-8')
s = movies['production_companies'].str.split('|').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'production'
movies = movies.join(s)
movies = movies[movies['production'].isin(["Universal Pictures", "Paramount Pictures"])]
movies.to_excel('only_companies.xlsx', encoding='utf-8')

### 已回答的问题

 + 电影类型是如何随着时代变化而变化的?
 + 环球影业和派拉蒙影业之间的数据指标有什么区别?
 + 和非小说改编的电影相比，基于小说改编的电影表现得怎么样?
  
### 提出的问题
 
 + 平均评分较高的前几部电影预算和收入和电影类别之间有什么关系？
 
 
 [利用Tabluea可视化](https://public.tableau.com/profile/silery#!/)